In [ ]:
# Cella 1: import base e path

import sys
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm  # per Black–Scholes

ROOT = Path("..").resolve()
SRC_PATH = ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.append(str(SRC_PATH))

from rbergomi.rbergomi_sde import RBergomiParams
from rbergomi.mc_pricer_rbergomi import price_call_rbergomi_mc

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["axes.grid"] = True

# Cella 2: Black–Scholes call price (benchmark)

def bs_call_price(S0, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return max(S0 - K, 0.0)
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Cella 3: parametri e griglia di strike

S0 = 100.0
T = 1.0
r = 0.0

params_rb = RBergomiParams(
    hurst=0.1,
    eta=2.0,
    rho=-0.7,
    xi0=0.04,
    r=r,
)

n_steps = 252
n_paths = 5000

strikes = np.linspace(60, 140, 9)
strikes

# Cella 4: calcolo prezzi rBergomi e Black–Scholes

rb_prices = []
bs_prices = []

sigma_bs = np.sqrt(params_rb.xi0)  # vol media di riferimento

for K in strikes:
    price_rb, se_rb = price_call_rbergomi_mc(
        S0=S0,
        K=K,
        T=T,
        n_steps=n_steps,
        n_paths=n_paths,
        params=params_rb,
        random_state=123,  # fisso per riproducibilità
    )
    rb_prices.append(price_rb)
    bs_prices.append(bs_call_price(S0, K, T, r, sigma_bs))

rb_prices = np.array(rb_prices)
bs_prices = np.array(bs_prices)

rb_prices, bs_prices

# Cella 5: confronto prezzi vs strike

plt.figure()
plt.plot(strikes, rb_prices, marker="o", label="rBergomi MC")
plt.plot(strikes, bs_prices, marker="x", linestyle="--", label="Black–Scholes")
plt.title("European Call Prices: rough Bergomi vs Black–Scholes")
plt.xlabel("Strike K")
plt.ylabel("Call price")
plt.legend()
plt.show()
